In [13]:
import numpy as np
import qiskit as qk
from qiskit import Aer
from qiskit.compiler import transpile, assemble
import time

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from samplers import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Looping over Circuits vs. Assembled Circuits Benchmark

In [14]:
backend = Aer.get_backend("qasm_simulator")

n_qubits = 4
repeats = 1000
shots = 100000

In [ ]:
np.random.seed(42)
start = time.time()
counts_list = []
for i in range(repeats):
    storage = qk.QuantumRegister(n_qubits)
    clas_reg = qk.ClassicalRegister(1)
    
    circuit = qk.QuantumCircuit(storage, clas_reg)
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])

    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])
        
    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
       
    circuit.measure(storage[-1], clas_reg)
    
    job = qk.execute(circuit, backend, shots=shots)
    counts = job.result().get_counts()
    counts_list.append(counts)

end = time.time()
print(end-start)

In [ ]:
print(sum(counts.values()))
for bitstring, samples in counts.items():
    print(bitstring, samples)

In [ ]:
np.random.seed(42)
start = time.time()
circuit_list = []
for i in range(repeats):
    storage = qk.QuantumRegister(n_qubits)
    clas_reg = qk.ClassicalRegister(1)

    circuit = qk.QuantumCircuit(storage, clas_reg)
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])

    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])
        
    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
    
    circuit.measure(storage[-1], clas_reg)
    
    circuit_list.append(circuit) 
    

transpiled_list = transpile(circuit_list, backend=backend)
qobject_list = assemble(transpiled_list, backend=backend, shots=shots, max_parallel_shots = 1, max_parallel_experiments = 0)
job = backend.run(qobject_list)

end = time.time()
print(end-start)

In [ ]:
count_list = []
for circuit in circuit_list:   
    count_list.append(job.result().get_counts(circuit))

### Stateverctor based circuit

In [17]:
backend = qk.providers.aer.StatevectorSimulator()

n_qubits = 4
repeats = 10000

In [18]:
np.random.seed(42)
start = time.time()
circuit_list = []
for i in range(repeats):
    storage = qk.QuantumRegister(n_qubits)

    circuit = qk.QuantumCircuit(storage)
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])

    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])
        
    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
        
    job = qk.execute(circuit, backend)
    statevector = job.result().get_statevector(circuit)

end = time.time()
print(end-start)

20.751400232315063


In [20]:
np.random.seed(42)
start = time.time()
circuit_list = []
for i in range(repeats):
    storage = qk.QuantumRegister(n_qubits)
    clas_reg = qk.ClassicalRegister(1)

    circuit = qk.QuantumCircuit(storage, clas_reg)
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])

    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
    
    for j in range(n_qubits):
        circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])
        
    for j in range(n_qubits - 1):
        circuit.cx(storage[j], storage[j + 1])
    
    circuit_list.append(circuit) 
    
end = time.time()
print(end-start)

#transpiled_list = transpile(circuit_list, backend=backend)
#end = time.time()
#print(end-start)

#qobject_list = assemble(circuit_list, backend=backend)
#end = time.time()
#print(end-start)

#job = backend.run(qobject_list)
#end = time.time()
#print(end-start)

for circuit in circuit_list:
    job = backend.run(circuit)
    job.result().get_statevector(circuit)

end = time.time()
print(end-start)

4.501038312911987


AttributeError: 'QuantumCircuit' object has no attribute 'config'

In [ ]:
#print(statevector_list)
print(output_list[:10])

### Multiple samples feed-forward layer

In [ ]:
x = np.array([[1,2,3], [4,5,6], [7,8,9], [10,11,12]])

In [ ]:
np.random.seed(42)
layer = QLayer(n_qubits=3, 
               n_features=3, 
               n_targets=2, 
               encoder=Encoder(), 
               ansatz=Ansatz(), 
               reps=1, 
               scale=np.pi, 
               backend=backend, 
               shots=100000)

In [ ]:
y = [layer(x_.reshape(1,-1)) for x_ in x]
print(y)

In [ ]:
y = layer(x)
print(y)

### Multiple samples backward

In [ ]:
x = np.array([[1,2,3], [4,5,6], [7,8,9], [10,11,12]], dtype="float64")
y = np.array([[1], [2], [3], [4]])

In [ ]:
np.random.seed(42)
layer1 = QLayer(n_qubits=3, 
                n_features=3, 
                n_targets=2, 
                encoder=Encoder(), 
                ansatz=Ansatz(), 
                reps=1, 
                scale=np.pi, 
                backend=backend, 
                shots=10000)

layer2 = QLayer(n_qubits=2, 
                n_features=2, 
                n_targets=1, 
                encoder=Encoder(), 
                ansatz=Ansatz(), 
                reps=1, 
                scale=np.pi, 
                backend=backend, 
                shots=10000)
layer = [layer1, layer2]

optimizer = Adam()

network = NeuralNetwork(layer, optimizer)

In [ ]:
print(network.predict(x))

In [ ]:
network.backward(x,y)

In [ ]:
print(network.weight_gradient_list)

In [ ]:
network.step()

### Statevector backend

In [ ]:
np.random.seed(42)

backend = Aer.get_backend("statevector_simulator")
n_qubits = 10
#shots=100

In [ ]:
storage = qk.QuantumRegister(n_qubits)
#clas_reg = qk.ClassicalRegister(1)

circuit = qk.QuantumCircuit(storage)

for j in range(n_qubits):
    circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])

for j in range(n_qubits - 1):
    circuit.cx(storage[j], storage[j + 1])

for j in range(n_qubits):
    circuit.ry(np.random.uniform(0, 2*np.pi), storage[j])

for j in range(n_qubits - 1):
    circuit.cx(storage[j], storage[j + 1])

job = qk.execute(circuit, backend)
counts = job.result().get_statevector()
print(counts)

In [ ]:
print(circuit)